<a href="https://colab.research.google.com/github/p-disha/ShopUNow-Agent/blob/main/ShopUNow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#########################################################################################


##Tech Stack

* LangChain / langchain_community-	Provides VectorStores (FAISS), Document abstraction, Embeddings, and Retrieval.


* FAISS (vectorstore)-	For embedding storage & similarity search (RAG).
Sentence-Transformers embeddings-	To convert document chunks into embedding vectors.


* **pdfminer.six + pytesseract + PIL**-	Extract text from PDFs, images (OCR) and markdown/text files — for building corpus.


* Markdownify	Convert markdown files to plain text.


* LangGraph (StateGraph etc.)-	The agent orchestration framework: state + nodes + transitions.


* Pydantic-	For structured schemas of state and tool inputs (validation, typing).


* LLM backends- OpenAI, Gemini (if available)	For synthesis / general LLM responses.

**Parses different document types (text, csv, pdf, image) into a corpus.**

**Chunks documents into manageable pieces using RecursiveCharacterTextSplitter.**

**Builds a FAISS index, persists it.**

**Sets up intent routing + tools for order status, returns, tickets.**

**Handles RAG retrieval + LLM synthesis with system prompt.**

**Passes retriever via RunnableConfig/configurable, avoiding earlier bug.**

**Good structure using StateGraph, Pydantic state schemas.**

In [10]:
# =========================
# Cell A – Setup & LLM with safer secret handling
# =========================

import os
import subprocess
import sys

# Detect whether we are running on Render / production
IS_RENDER = bool(os.getenv("PORT") or os.getenv("RENDER_SERVICE_TYPE"))

# --- Dev-only dependency install (Colab / local), skipped on Render ---
if not IS_RENDER:
    deps = [
        "langchain_community",
        "faiss-cpu",
        "langchain-openai",
        "langchain-google-genai",
        "pydantic",
        "typing_extensions",
        "vaderSentiment",
        "langgraph",
        "rapidfuzz",
        "flask",
        "flask-cors",
        "pyngrok",
    ]
    try:
        subprocess.run([sys.executable, "-m", "pip", "install", "-qU", *deps], check=False)
    except Exception as ex:
        print("⚠️ Dev install failed:", ex)

# --- Secrets / API key handling ---
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GEMINI_API_KEY = os.getenv("GOOGLE_API_KEY")

if not IS_RENDER:
    # In dev/Colab, attempt to fetch from google.colab userdata
    try:
        from google.colab import userdata  # type: ignore
        if not OPENAI_API_KEY:
            OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
        if not GEMINI_API_KEY:
            GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")
        # If found, set them to env so downstream code sees them
        if OPENAI_API_KEY:
            os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
        if GEMINI_API_KEY:
            os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY
    except Exception:
        # Not in Colab or userdata not available
        pass

# In production / Render, require at least one key
if IS_RENDER:
    assert OPENAI_API_KEY or GEMINI_API_KEY, "In production, you must set OPENAI_API_KEY or GOOGLE_API_KEY."

# --- Minimal setup only (no heavy model or FAISS initialization here) ---

# Save model name or config for later initialization
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"

def get_chat_model():
    """
    Return a chat model (Gemini or OpenAI or fallback).
    Delay imports to here to avoid import-time costs.
    """
    if GEMINI_API_KEY:
        try:
            from langchain_google_genai import ChatGoogleGenerativeAI
            return ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.2)
        except Exception as e:
            print("Gemini init failed:", e, flush=True)

    if OPENAI_API_KEY:
        try:
            from langchain_openai import ChatOpenAI
            return ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
        except Exception as e:
            print("OpenAI init failed:", e, flush=True)

    # Fallback dummy (for dev/edge case)
    from langchain_core.messages import HumanMessage
    class _Mock:
        def invoke(self, messages):
            last = None
            for m in reversed(messages):
                if isinstance(m, HumanMessage):
                    last = m
                    break
            return type("Resp", (), {"content": "[MOCK] " + (last.content if last else "")})
    print("Using mock LLM fallback")
    return _Mock()

# `LLM` will be set later once backend is initialized
LLM = None

SYSTEM_POLICY = (
    "You are ShopUNow Assistant. Be concise and accurate. Use the internal knowledge base when possible. "
    "If unable to answer, ask a clarifying question."
)

print("Cell A (setup) loaded — heavy initialization delayed.")


Cell A (setup) loaded — heavy initialization delayed.


In [11]:
# =========================
# Cell A.1 – Lazy Load FAQ Dataset & Build Vector Store
# =========================

import os
import json
import faiss
from typing import List, Tuple
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_openai import OpenAIEmbeddings

# --- Global cache to avoid reloading repeatedly ---
_FAQ_VECTOR_STORE = None
_FAQ_DOCS: List[Document] = []
_FAQ_PATH = None


def resolve_faq_path() -> str:
    """
    Resolve path to the FAQ JSONL file.
    Supports Colab (/content), data/, or root directory.
    """
    candidates = [
        "/content/shopunow_faqs.jsonl",
        os.path.join(os.getcwd(), "data", "shopunow_faqs.jsonl"),
        os.path.join(os.getcwd(), "shopunow_faqs.jsonl"),
    ]
    for c in candidates:
        if os.path.exists(c):
            return c
    raise FileNotFoundError("❌ shopunow_faqs.jsonl not found in common paths.")


def load_faq_documents(path: str) -> List[Document]:
    """
    Read JSONL file and parse into LangChain Documents.
    """
    docs = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                record = json.loads(line)
                question = record.get("question", "").strip()
                answer = record.get("answer", "").strip()
                dept = record.get("department", "unknown").strip()
                if not question or not answer:
                    continue
                combined_text = f"Q: {question}\nA: {answer}"
                docs.append(
                    Document(
                        page_content=combined_text,
                        metadata={
                            "department": dept,
                            "question": question,
                            "answer": answer,
                        },
                    )
                )
            except json.JSONDecodeError as e:
                print(f"⚠️ Skipping invalid JSON line: {e}")
    return docs


def build_faq_vector_store(docs: List[Document]) -> Tuple[FAISS, List[Document]]:
    """
    Build FAISS vector store with normalized embeddings.
    """
    if not docs:
        raise ValueError("No FAQ documents to build vector store.")
    embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")
    dim = len(embedding_model.embed_query("hello world"))
    index = faiss.IndexFlatIP(dim)
    store = FAISS(
        embedding_function=embedding_model,
        index=index,
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
    )
    ids = [f"faq_{i+1}" for i in range(len(docs))]
    store.add_documents(docs, ids=ids)
    return store, docs


def get_faq_vector_store() -> Tuple[FAISS, List[Document]]:
    """
    Lazy initializer for the FAQ vector store.
    Returns cached instance if already built.
    """
    global _FAQ_VECTOR_STORE, _FAQ_DOCS, _FAQ_PATH
    if _FAQ_VECTOR_STORE is not None:
        return _FAQ_VECTOR_STORE, _FAQ_DOCS

    try:
        _FAQ_PATH = resolve_faq_path()
        _FAQ_DOCS = load_faq_documents(_FAQ_PATH)
        _FAQ_VECTOR_STORE, _FAQ_DOCS = build_faq_vector_store(_FAQ_DOCS)
        dept_set = {d.metadata.get("department", "unknown") for d in _FAQ_DOCS}
        print(f"✅ Built vector store with {len(_FAQ_DOCS)} FAQs across {len(dept_set)} departments")
    except Exception as e:
        print(f"❌ Error initializing FAQ store: {e}")
        raise

    return _FAQ_VECTOR_STORE, _FAQ_DOCS


In [12]:
# =========================
# Cell B — Agent (lazy FAQ store, cosine/IP thresholds, escalation)
# =========================

import os
import json
import numpy as np
import random
from typing import Optional, List, Dict, Any, Literal, Tuple
from pydantic import BaseModel, Field
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from rapidfuzz import fuzz

# LangChain base types
from langchain_core.documents import Document

# 👇 IMPORTANT: uses the lazy loader you created in Cell A.1
# make sure Cell A.1 (with get_faq_vector_store) ran before this cell
# It returns (FAISS_store, docs_list) and caches internally.
from __main__ import get_faq_vector_store  # if this code is in one notebook
# If this is split across files, replace with:
# from your_module import get_faq_vector_store

# --------------------
# Determinism (for reproducibility)
# --------------------
random.seed(42)
np.random.seed(42)

# --------------------
# Sentiment (lazy init)
# --------------------
_sentiment_analyzer: Optional[SentimentIntensityAnalyzer] = None

def detect_sentiment(text: str) -> Literal["negative", "neutral", "positive"]:
    global _sentiment_analyzer
    if _sentiment_analyzer is None:
        _sentiment_analyzer = SentimentIntensityAnalyzer()
    if not text:
        return "neutral"
    c = _sentiment_analyzer.polarity_scores(text).get("compound", 0.0)
    if c <= -0.3:
        return "negative"
    if c >= 0.3:
        return "positive"
    return "neutral"

# --------------------
# Dept classifier with confidence & tie handling
# --------------------
DEPT_KEYWORDS: Dict[str, List[str]] = {
    "Orders & Returns": [
        "order", "order status", "track order", "tracking", "shipment",
        "delivery", "package", "where is my order", "cancel order",
        "return", "refund", "replace", "exchange", "pickup"
    ],
    "Payments & Billing": [
        "payment", "upi", "card", "wallet", "cod", "invoice", "coupon",
        "billing", "charged", "charge", "emi", "price", "gst"
    ],
    "Customer Support": [
        "support", "contact", "help", "issue", "complaint", "agent",
        "human", "speak to", "phone", "call", "email", "hours", "timings"
    ],
    "HR & IT Helpdesk": [
        "password", "vpn", "access", "onboarding", "hardware", "software",
        "leave", "policy", "salary", "payroll", "payslip", "stipend",
        "salary date", "pay date", "salary delayed", "hrms", "hr portal"
    ],
}

def classify_department_with_confidence(user_query: str) -> Tuple[Optional[str], float, Dict[str, int]]:
    text = (user_query or "").lower()
    scores: Dict[str, int] = {}
    for dept, kws in DEPT_KEYWORDS.items():
        score = sum(1 for kw in kws if kw in text)
        scores[dept] = score

    sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    top_dept, top_score = sorted_scores[0]

    if top_score == 0:
        return None, 0.0, scores

    # near-tie / tie guard (strict here: if second >= top, treat as ambiguous)
    if len(sorted_scores) > 1 and sorted_scores[1][1] >= top_score:
        return None, 0.0, scores

    conf = 0.6 if top_score == 1 else (0.75 if top_score == 2 else 0.9)
    return top_dept, conf, scores

# --------------------
# Similarity thresholds (IP/cosine)
# --------------------
DEPT_SIM_THRESHOLDS = {
    "Orders & Returns": 0.80,
    "Payments & Billing": 0.78,
    "Customer Support": 0.75,
    "HR & IT Helpdesk": 0.80,
    None: 0.80,  # fallback
}

# --------------------
# Agent state
# --------------------
class AgentState(BaseModel):
    user_input: str
    department: Optional[str] = None
    dept_confidence: float = 0.0
    sentiment: Optional[Literal["negative","neutral","positive"]] = None
    tools_used: List[str] = Field(default_factory=list)
    retrieved: List[Dict[str, Any]] = Field(default_factory=list)
    intent: Optional[Literal["rag","order_status","return_create","ticket","human_escalation","unknown"]] = None
    answer: Optional[str] = None
    confidence: float = 0.0   # overall answer confidence
    reason: Optional[str] = None  # why we escalated / confidence is low

# --------------------
# Helpers
# --------------------
def extract_answer_text(page_content: str) -> str:
    """If content is 'Q: ...\\nA: ...', return only the A: part."""
    if not page_content:
        return page_content
    lower = page_content.lower()
    if "a:" in lower:
        idx = lower.find("a:")
        return page_content[idx+2:].strip().lstrip(":").strip()
    if page_content.strip().startswith("Q:"):
        return page_content.replace("Q:", "", 1).strip()
    return page_content

def contains_any(text: str, keywords: List[str]) -> bool:
    low = text.lower()
    return any(kw in low for kw in keywords)

# --------------------
# Routing
# --------------------
def route_intent(state: AgentState) -> Dict[str, Any]:
    user_query = state.user_input or ""
    ql = user_query.lower()

    sentiment = detect_sentiment(user_query)
    dept, dept_conf, _scores = classify_department_with_confidence(user_query)

    if sentiment == "negative":
        intent = "human_escalation"
    elif contains_any(ql, ["order status", "track order", "where is my order", "tracking", "shipment", "delivery", "package"]):
        intent = "order_status"
    elif contains_any(ql, ["return", "refund", "replace", "exchange"]):
        intent = "rag" if contains_any(ql, ["policy", "how many", "days", "window"]) else "return_create"
    elif contains_any(ql, ["ticket", "helpdesk", "support issue", "complaint", "problem"]):
        intent = "ticket"
    else:
        intent = "rag"

    print(f"[route_intent] input={user_query!r} -> intent={intent}, dept={dept}, dept_conf={dept_conf:.2f}, sentiment={sentiment}")
    return {"intent": intent, "department": dept, "dept_confidence": dept_conf, "sentiment": sentiment}

# --------------------
# Tool node (does lazy FAQ load on demand)
# --------------------
def _filter_by_department(results: List[Any], predicted_dept: Optional[str]) -> List[Any]:
    if not results or not predicted_dept:
        return results or []
    filtered = [(doc, score) for doc, score in results if (doc.metadata or {}).get("department") == predicted_dept]
    return filtered or results

def tool_node(state: AgentState) -> Dict[str, Any]:
    intent = state.intent
    user_query = (state.user_input or "").strip()
    predicted_department = state.department
    dept_conf = state.dept_confidence
    print(f"[tool_node] intent={intent}, dept={predicted_department}, dept_conf={dept_conf:.2f}, input={user_query!r}")

    # --- Direct tools ---
    if intent == "order_status":
        has_order_id = any(tok.startswith(("ORD-", "ord-")) or tok.isdigit() for tok in user_query.replace("#", " ").split())
        if not has_order_id:
            return {
                "answer": "To check a specific order, please share your Order ID (e.g., ORD-1234).",
                "tools_used": ["order_status_tool"],
                "confidence": 0.65,
                "reason": "order_id_missing",
            }
        return {
            "answer": "Your order is being processed and will be shipped soon.",
            "tools_used": ["order_status_tool"],
            "confidence": 0.9,
        }

    if intent == "return_create":
        return {
            "answer": "Return initiated. You will receive pickup and label details via email.",
            "tools_used": ["return_create_tool"],
            "confidence": 0.9,
        }

    if intent == "ticket":
        return {
            "answer": "A support ticket has been created. Someone will get back to you shortly.",
            "tools_used": ["ticket_tool"],
            "confidence": 0.85,
        }

    if intent == "human_escalation":
        return {
            "answer": "I’m sorry for the inconvenience. Escalating to human support — someone will reach out to you soon.",
            "tools_used": ["escalation"],
            "confidence": 0.2,
            "reason": "negative_sentiment",
        }

    # --- Retrieval (RAG) ---
    if intent == "rag":
        if not predicted_department or dept_conf < 0.6:
            return {
                "answer": "Your query could relate to multiple areas. I’m escalating to human support to ensure it’s handled correctly.",
                "tools_used": ["escalation"],
                "confidence": 0.2,
                "reason": "low_department_confidence",
            }

        # Lazy load the FAQ store only when needed
        try:
            faq_vector_store, faq_documents = get_faq_vector_store()
        except Exception as e:
            print(f"[tool_node] ❌ Could not initialize FAQ store: {e}")
            return {
                "answer": "Something went wrong while preparing the knowledge base. Escalating to human support.",
                "tools_used": ["escalation"],
                "confidence": 0.0,
                "reason": "faq_store_init_error",
            }

        try:
            results = faq_vector_store.similarity_search_with_score(user_query, k=5)
            results = [(doc, score) for doc, score in results if doc is not None]
            results = _filter_by_department(results, predicted_department)

            if not results:
                return {
                    "answer": "Sorry, I couldn’t find reliable information in our knowledge base. Escalating to human support.",
                    "tools_used": ["escalation"],
                    "confidence": 0.2,
                    "reason": "no_results",
                }

            top_doc, sim = results[0]
            print(f"[tool_node] Top similarity={sim:.4f}")
            sim_threshold = DEPT_SIM_THRESHOLDS.get(predicted_department, DEPT_SIM_THRESHOLDS[None])

            if sim < sim_threshold:
                # Fuzzy question fallback if similarity low
                best_doc, best_fuzzy = None, 0.0
                q_low = user_query.lower()
                for doc in faq_documents:
                    fs = fuzz.partial_ratio(q_low, doc.metadata.get("question", "").lower()) / 100.0
                    if fs > best_fuzzy:
                        best_doc, best_fuzzy = doc, fs
                if best_doc and best_fuzzy >= 0.92:
                    dept_meta = best_doc.metadata.get("department", "unknown")
                    clean_answer = extract_answer_text(best_doc.page_content)
                    return {
                        "answer": f"{clean_answer} (Dept: {dept_meta})",
                        "tools_used": ["rag_fuzzy_fallback"],
                        "retrieved": [{
                            "question": best_doc.metadata.get("question", ""),
                            "answer": clean_answer,
                            "fuzzy_score": float(best_fuzzy),
                            "source": dept_meta
                        }],
                        "confidence": float(min(0.85, best_fuzzy)),
                        "reason": "fuzzy_match_high",
                    }
                return {
                    "answer": "I’m not fully confident about the answer. Escalating to human support.",
                    "tools_used": ["escalation"],
                    "confidence": float(sim),
                    "reason": "low_similarity",
                }

            # Good similarity → return clean answer
            dept_meta = (top_doc.metadata or {}).get("department", "unknown")
            clean_answer = extract_answer_text(top_doc.page_content)
            return {
                "answer": f"{clean_answer} (Dept: {dept_meta})",
                "tools_used": ["rag_retrieval"],
                "retrieved": [{
                    "question": top_doc.metadata.get("question", ""),
                    "answer": clean_answer,
                    "similarity": float(sim),
                    "source": dept_meta
                }],
                "confidence": float(sim),
            }

        except Exception as e:
            print(f"[tool_node] ❌ Retrieval error: {e}")
            return {
                "answer": "Something went wrong while searching. Escalating to human support.",
                "tools_used": ["escalation"],
                "confidence": 0.0,
                "reason": "retrieval_exception",
            }

    # Fallback
    return {
        "answer": "Could you please rephrase your request?",
        "tools_used": ["fallback"],
        "confidence": 0.3,
        "reason": "fallback",
    }

# --------------------
# Synthesis (no-op — kept for future expansion)
# --------------------
def synthesis_node(state: AgentState) -> Dict[str, Any]:
    return {}

# --------------------
# Build Graph (lightweight — no FAISS work happens here)
# --------------------
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

graph = StateGraph(AgentState)
graph.add_node("route", route_intent)
graph.add_node("tool", tool_node)
graph.add_node("synth", synthesis_node)

graph.add_edge(START, "route")
graph.add_edge("route", "tool")
graph.add_edge("tool", "synth")
graph.add_edge("synth", END)

memory = MemorySaver()
app = graph.compile(checkpointer=memory)

# --------------------
# Ask wrapper
# --------------------
def ask(user_query: str, thread_id: Optional[str] = None) -> str:
    if not thread_id:
        import uuid
        thread_id = f"thread_{uuid.uuid4().hex}"
    out = app.invoke(
        {"user_input": user_query},
        config={"configurable": {"thread_id": thread_id}},
    )
    return out.get("answer", "No answer generated.")


In [13]:
# =========================
# Cell B.1 - Testing (Improved Output & Meaningful Names)
# =========================

import os
import uuid
from pprint import pprint

# Guard: only run this in dev / notebook environments, not in production (Render)
if os.getenv("RENDER_SERVICE_TYPE") or os.getenv("PORT"):
    print("[Skip] Testing cell running in production environment.")
else:
    test_queries = [
        "What are your support hours?",
        "Tell me order status for order id ORD-1234",
        "I want a return because the product is wrong",
        "My password reset isn't working, this is frustrating",
        "I submitted a complaint about a support issue",
        "How do I pay with UPI?",
        "How to apply for leaves?",
        "what is the leave policy?",
        "where is my order?",
        "my order is delayed",
        "whats the return polcy? how many days can i return the product?",
        "how many leaves can I take?",
        " how many days for rturn?",
        "my retturn is delayed",
        "Please replace my shirt size",
        "where is my order",
        "when will i get my salary",
        "my salaray is delayed",
        "I need help,",  # ambiguous
    ]

    for user_query in test_queries:
        conversation_id = f"conv_{uuid.uuid4().hex}"
        try:
            state = AgentState(user_input=user_query)
            route_info = route_intent(state)
            response = app.invoke(
                {"user_input": user_query},
                config={"configurable": {"thread_id": conversation_id}}
            )

            print("=" * 80)
            print(f"🧑 User Query: {user_query}")
            print(f"➡️ Intent: {route_info.get('intent')} | Dept: {route_info.get('department')} | Sentiment: {route_info.get('sentiment')}")
            print(f"🤖 Agent Answer: {response.get('answer', '⚠️ No answer')}")
            print(f"🛠️ Tools Used: {response.get('tools_used')}")
            if response.get("retrieved"):
                print("📄 Retrieved Context:")
                pprint(response["retrieved"])
            print("=" * 80 + "\n")
        except Exception as e:
            print(f"Error handling test query {user_query!r}: {e}")
            traceback.print_exc()


[route_intent] input='What are your support hours?' -> intent=rag, dept=Customer Support, dept_conf=0.75, sentiment=positive
[route_intent] input='What are your support hours?' -> intent=rag, dept=Customer Support, dept_conf=0.75, sentiment=positive
[tool_node] intent=rag, dept=Customer Support, dept_conf=0.75, input='What are your support hours?'
✅ Built vector store with 119 FAQs across 5 departments
[tool_node] Top similarity=0.8337
🧑 User Query: What are your support hours?
➡️ Intent: rag | Dept: Customer Support | Sentiment: positive
🤖 Agent Answer: Yes, live chat is available from 9 AM to 9 PM IST on our website and mobile app. (Dept: Customer Support)
🛠️ Tools Used: ['rag_retrieval']
📄 Retrieved Context:
[{'answer': 'Yes, live chat is available from 9 AM to 9 PM IST on our website '
            'and mobile app.',
  'question': 'Do you have live chat support?',
  'similarity': 0.8337283134460449,
  'source': 'Customer Support'}]

[route_intent] input='Tell me order status for ord

In [14]:
# =========================
# Cell C — Flask API (Render + Colab Compatible, 10-min safe)
# =========================
import os
import sys
import traceback
import threading
import uuid
import socket
import time
from flask import Flask, request, jsonify
from flask_cors import CORS

# --- Flask setup ---
flask_app = Flask(__name__)
CORS(flask_app)

def _debug(msg: str):
    """Safe logger for Colab/Render."""
    print(msg, flush=True)

# --------------------
# Agent bridge
# --------------------
def call_agent(query: str) -> str:
    """Route to active agent or fallback."""
    if "ask" in globals() and callable(globals()["ask"]):
        _debug("[AGENT] Using ask()")
        return globals()["ask"](query)
    for name in ["agent_app", "graph_app", "app"]:
        obj = globals().get(name)
        if hasattr(obj, "invoke"):
            _debug(f"[AGENT] Using graph '{name}'.invoke()")
            cfg = {"configurable": {"thread_id": f"api-{uuid.uuid4().hex}"}}
            out = obj.invoke({"user_input": query}, config=cfg)
            return out.get("answer", "No answer generated.")
    return "⚠️ No active agent found — please check initialization."

# --------------------
# Routes
# --------------------
@flask_app.route("/ask", methods=["POST", "GET"])
def ask_api():
    try:
        _debug("[API] /ask endpoint hit")
        if request.method == "POST":
            data = request.get_json(silent=True) or {}
            query = (data.get("query") or "").strip()
        else:
            query = (request.args.get("query") or "").strip()

        if not query:
            return jsonify({"error": "Empty query"}), 400

        _debug(f"[API] Query: {query!r}")
        answer = call_agent(query)
        _debug(f"[API] Answer: {answer!r}")

        return jsonify({"query": query, "answer": answer})
    except Exception as e:
        _debug("[API] ❌ Exception:")
        traceback.print_exc(file=sys.stdout)
        return jsonify({"error": "Internal Server Error", "details": str(e)}), 500

@flask_app.route("/", methods=["GET"])
def home():
    return jsonify({"status": "ok", "message": "ShopUNow backend active"})

# --------------------
# Environment detection
# --------------------
def _get_colab_secret(name: str):
    try:
        from google.colab import userdata
        return userdata.get(name)
    except Exception:
        return None

def find_free_port(default=5000):
    """Find a free TCP port."""
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(("", 0))
        return s.getsockname()[1]

IS_RENDER = bool(os.getenv("RENDER_SERVICE_TYPE") or os.getenv("PORT"))
NGROK_AUTH_TOKEN = _get_colab_secret("NGROK_AUTH_TOKEN")

# =================================================
# 🔹 COLAB DEV MODE (auto ngrok, background thread)
# =================================================
if not IS_RENDER and NGROK_AUTH_TOKEN:
    PORT = find_free_port(5000)
    _debug(f"▶️ [Colab Mode] Launching Flask on port {PORT}")

    def run_flask():
        flask_app.run(host="0.0.0.0", port=PORT, debug=False, use_reloader=False)
    threading.Thread(target=run_flask, daemon=True).start()

    try:
        from pyngrok import ngrok
    except ImportError:
        import subprocess
        subprocess.run([sys.executable, "-m", "pip", "install", "-q", "pyngrok"], check=False)
        from pyngrok import ngrok

    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    try:
        ngrok.kill()
    except Exception:
        pass

    try:
        _debug(f"🌐 Opening ngrok tunnel on port {PORT} …")
        tunnel = ngrok.connect(PORT, bind_tls=True)
        public_url = getattr(tunnel, "public_url", str(tunnel))
        _debug(f"🚀 Backend Public URL: {public_url}")

        with open("/content/backend_url.txt", "w") as f:
            f.write(public_url.strip())
        _debug("📁 backend_url.txt saved for frontend sync")

        print("\n✅ Test locally:")
        print(f'curl -X POST "{public_url}/ask" -H "Content-Type: application/json" -d \'{{"query": "Hello"}}\'')
    except Exception as e:
        _debug(f"❌ ngrok setup failed: {e}")
        traceback.print_exc(file=sys.stdout)

# =================================================
# 🔹 RENDER PROD MODE (Gunicorn-managed)
# =================================================
else:
    PORT = int(os.environ.get("PORT", 8000))
    _debug(f"▶️ [Render Mode] Binding Flask to 0.0.0.0:{PORT}")

    # Health wait (up to 10 min)
    for i in range(600):  # 600s = 10min
        try:
            _debug(f"[Startup Check] Iteration {i}/600 ...")
            time.sleep(1)
        except KeyboardInterrupt:
            break

    # Gunicorn handles process launch — no manual .run()
    if __name__ == "__main__":
        flask_app.run(host="0.0.0.0", port=PORT, debug=False)


▶️ [Colab Mode] Launching Flask on port 38901
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:38901
 * Running on http://172.28.0.12:38901
INFO:werkzeug:Press CTRL+C to quit


🌐 Opening ngrok tunnel on port 38901 …
🚀 Backend Public URL: https://68069f4b914f.ngrok-free.app
📁 backend_url.txt saved for frontend sync

✅ Test locally:
curl -X POST "https://68069f4b914f.ngrok-free.app/ask" -H "Content-Type: application/json" -d '{"query": "Hello"}'


In [16]:
# =========================
# Cell D — Streamlit Frontend (Colab Dev Only; Skipped in Render)
# =========================
import os
import sys
import subprocess
import threading
import time

# -----------------------------
# 🚫 Skip frontend in Render
# -----------------------------
if os.getenv("RENDER_SERVICE_TYPE") or os.getenv("PORT"):
    print("[Render Mode] Skipping Streamlit frontend.")
else:
    print("▶️ [Colab Dev] Initializing Streamlit frontend…")

    # -----------------------------
    # ✅ Ensure dependencies
    # -----------------------------
    try:
        import streamlit
        import requests
        import pyngrok
    except ImportError:
        print("📦 Installing required frontend packages…")
        deps = ["streamlit", "requests", "pyngrok"]
        subprocess.run([sys.executable, "-m", "pip", "install", "-qU", *deps], check=False)
        import streamlit
        import requests
        import pyngrok

    # -----------------------------
    # 🧠 Detect backend URL
    # -----------------------------
    backend_url_file = "/content/backend_url.txt"
    default_api_url = "http://127.0.0.1:5000/ask"

    if os.path.exists(backend_url_file):
        try:
            with open(backend_url_file, "r") as f:
                url = f.read().strip()
            if url:
                default_api_url = url.rstrip("/") + "/ask"
                print(f"✅ Backend URL found: {default_api_url}")
        except Exception as e:
            print(f"⚠️ Could not read backend_url.txt: {e}")
    else:
        print(f"⚠️ backend_url.txt not found — using {default_api_url}")

    # -----------------------------
    # 🖋️ Generate Streamlit App
    # -----------------------------
    app_code = f"""
import streamlit as st
import requests

st.set_page_config(page_title="ShopUNow Agent", layout="centered")
st.title("🛍️ ShopUNow AI Assistant")

api_url = st.sidebar.text_input("Flask API URL", value="{default_api_url}")
st.sidebar.caption("Update this if you have a different backend URL")

st.divider()
st.subheader("💬 Chat Interface")

if "chat" not in st.session_state:
    st.session_state.chat = []

query = st.text_input("Enter your question:")

if st.button("Ask"):
    if query.strip():
        st.session_state.chat.append(("🧑 You", query))
        try:
            resp = requests.post(api_url, json={{"query": query}}, timeout=25)
            if resp.status_code == 200:
                ans = resp.json().get("answer", "⚠️ No answer received.")
            else:
                ans = f"⚠️ HTTP {{resp.status_code}}: {{resp.text}}"
        except Exception as e:
            ans = f"⚠️ Request failed: {{e}}"
        st.session_state.chat.append(("🤖 Agent", ans))

for sender, msg in st.session_state.chat:
    st.markdown(f"**{{sender}}:** {{msg}}")
"""

    with open("app_frontend.py", "w", encoding="utf-8") as f:
        f.write(app_code.strip())

    # -----------------------------
    # 🚀 Launch Streamlit (background)
    # -----------------------------
    def run_streamlit():
        print("▶️ Starting Streamlit frontend on port 8501…")
        subprocess.run([
            sys.executable, "-m", "streamlit", "run", "app_frontend.py",
            "--server.headless", "true",
            "--server.port", "8501",
            "--browser.gatherUsageStats", "false"
        ])

    threading.Thread(target=run_streamlit, daemon=True).start()
    time.sleep(5)

    # -----------------------------
    # 🌍 ngrok tunnel
    # -----------------------------
    try:
        from pyngrok import ngrok
        print("🌐 Launching ngrok tunnel for Streamlit (port 8501)…")
        tunnel = ngrok.connect(8501, bind_tls=True)
        public_url = getattr(tunnel, "public_url", str(tunnel))
        print(f"🚀 Streamlit Public URL: {public_url}")
        print("\n✅ Test your frontend:")
        print(f"{public_url}")
    except Exception as e:
        print(f"⚠️ Could not start ngrok tunnel: {e}")


▶️ [Colab Dev] Initializing Streamlit frontend…
✅ Backend URL found: https://68069f4b914f.ngrok-free.app/ask
▶️ Starting Streamlit frontend on port 8501…
🌐 Launching ngrok tunnel for Streamlit (port 8501)…
🚀 Streamlit Public URL: https://7cbfc54ed1ed.ngrok-free.app

✅ Test your frontend:
https://7cbfc54ed1ed.ngrok-free.app
